### Importing Libraries

In [1]:
#Importing libraries
import numpy as np
import pandas as pd
import cv2 
import pytesseract as pt
from PIL import Image
pt.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract"
import nltk
import spacy

In [2]:
import spacy
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to C:\Users\ROSHINI R
[nltk_data]     S\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\ROSHINI R
[nltk_data]     S\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ROSHINI R S\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter
from heapq import nlargest

### OCR

In [4]:
#OCR
def img_txt(img_path):
    global text
    img = cv2.imread(img_path)
    img = cv2.resize(img, None, fx = 1,fy = 1)
    kernel = np.ones((1, 1), np.uint8)
    img = cv2.dilate(img, kernel, iterations=1)
    img = cv2.erode(img, kernel, iterations=1)
    blurred=cv2.GaussianBlur(img,(5,5),0)
    edged=cv2.Canny(blurred,30,50)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    adpt = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY, 85,10)
    text = pt.image_to_string(adpt)
    text = text.replace('\n',' ')
    print(text)
    cv2.imshow("adaptive threshold",adpt)
    cv2.waitKey(0)

### Geting Input from the user

In [5]:
#inupt image through webcam
cam = cv2.VideoCapture(0)

cv2.namedWindow("Capture")

img_counter = 0

while True:
    ret, frame = cam.read()
    if not ret:
        print("failed to grab frame")
        break
    
    cv2.imshow("Capture", frame)
 
    k = cv2.waitKey(1)
    if k%256 == 27:
        # ESC pressed
        print("Escape hit, closing...")
        break
    elif k%256 == 32:
        # SPACE pressed  
        img_name = "opencv_frame_{}.png".format(img_counter)
        cv2.imwrite(img_name, frame)
        print("{} written!".format(img_name))
        img_counter += 1
    
cam.release()
cv2.destroyAllWindows()

Escape hit, closing...


In [7]:
#Load the image from local file
img_name = 'sample_image.jpg'

### Display raw text from the Image

In [8]:
#Process the image and print text
print("Scanned text:\n")
img_txt(img_name)

Scanned text:

Arti  ‘ial Intelligence And Machine Learning  Artificial Intelligence (Al) has gained widespread popularity in recent years, with its application flooding every business sector. Al has gained astonishingly great acceptance in the world of portable technology, by making various functions available at your fingertips. The fast speed of Al development, and accomplishments in automation, automated vehicles, the capacity to beat people at mind games, and computerized user support mean that Al is a progressive technology that will receive extraordinary rewards over the long haul.  The ongoing Al environment comprises robotics, machine learning (ML), and artificial neural networks (ANNs) that are improving the semantic, numerical, and legitimate thinking skills of any framework using Al. The two main variables driving the quick reception of Al are top caliber, versatile learning models, and the need to process a lot of information at reasonable costs. Very much like every other

### Summarize the text

### Using Spacy to summarise the text

In [9]:
nlp = spacy.load("en")
doc = nlp(text)
# len(list(doc.sents))

keyword = []
stopwords = list(STOP_WORDS)
pos_tag = ['PROPN', 'ADJ', 'NOUN', 'VERB']
for token in doc:
    if(token.text in stopwords or token.text in punctuation):
        continue
    if(token.pos_ in pos_tag):
        keyword.append(token.text)
        
freq_word = Counter(keyword)
print(freq_word.most_common(5))

OSError: [E941] Can't find model 'en'. It looks like you're trying to load a model from a shortcut, which is obsolete as of spaCy v3.0. To load the model, use its full name instead:

nlp = spacy.load("en_core_web_sm")

For more details on the available models, see the models directory: https://spacy.io/models. If you want to create a blank model, use spacy.blank: nlp = spacy.blank("en")

In [11]:
max_freq = Counter(keyword).most_common(1)[0][1]
for word in freq_word.keys():  
        freq_word[word] = (freq_word[word]/max_freq)
freq_word.most_common(5)

NameError: name 'keyword' is not defined

In [12]:
sent_strength={}
for sent in doc.sents:
    for word in sent:
        if word.text in freq_word.keys():
            if sent in sent_strength.keys():
                sent_strength[sent]+=freq_word[word.text]
            else:
                sent_strength[sent]=freq_word[word.text]
# print(sent_strength)

NameError: name 'doc' is not defined

In [13]:
summarized_sentences = nlargest(4, sent_strength, key=sent_strength.get)
# print(summarized_sentences)

In [14]:
final_sentences = [ w.text for w in summarized_sentences ]
summary = ' '.join(final_sentences)
print("Summarized text: \n")
print(summary)

Summarized text: 




### Extracting key phrases from the summary

In [15]:
import RAKE
import operator
stop_dir = r"C:\Users\ROSHINI R S\anaconda3\Lib\site-packages\RAKE\stoplists\SmartStopList.txt"
rake_object=RAKE.Rake(stop_dir)
def Sort_Tuple(tup):
    tup.sort(key=lambda x: x[1])
    return tup
keywords = Sort_Tuple(rake_object.run(summary))[-5:]
print("Keywords: \n",keywords)

ModuleNotFoundError: No module named 'RAKE'

In [14]:
Phrase=[]
for i in keywords:
    Phrase.append(i[0])
print("Keyphrases: ")
Phrase

Keyphrases: 


['chess-playing computers',
 'accomplish specific tasks',
 'self-driving cars',
 'natural language processing',
 'processing large amounts']

### Finding relevant links from the keywords

In [47]:
#Automate google search for getting relevant links
import requests
import urllib.request
import re

try:
    from googlesearch import search
except ImportError:
    print("No module named 'google' found")
Website_link=[]
website_video_link=[]
youtube_link=[]
news_link=[]

for i in Phrase:
    for j in search(i, tld="co.in", num=1, start=0,stop=1,pause=2,safe='on'):
        Website_link.append(j)
    for k in search(i, tld="co.in", num=1, start=0,stop=1,pause=2,tpe='vid',safe='on'):
        website_video_link.append(k)
    for l in search(i, tld="co.in", num=1, start=0,stop=1,pause=2,tpe='nws',safe='on'):
        news_link.append(l)

for m in Phrase:
  query=m.replace(" ","+")
  html=urllib.request.urlopen("https://www.youtube.com/results?search_query="+query)
  video_ids = re.findall(r"watch\?v=(\S{11})", html.read().decode())
  link=("https://www.youtube.com/watch?v=" + video_ids[0])
  youtube_link.append(link)


In [48]:
df = pd.DataFrame(list(zip(Phrase,Website_link,website_video_link,youtube_link,news_link)),columns = ['Key phrase', 'Website Link','website with video link','youtube video link','news link'])
print(df)

                                            Keywords  \
0  existing speech conversion apps allows live co...   
1  existing speech conversion applications conver...   
2                      convert old speech recordings   

                                              Link 1  \
0  https://www.researchgate.net/publication/25203...   
1        https://www.youtube.com/watch?v=6lLSx0nUxaE   
2      https://en.wikipedia.org/wiki/Registered_mail   

                                              Link 2  \
0  https://www.techradar.com/in/news/best-speech-...   
1  https://www.researchgate.net/publication/31248...   
2  https://www.rev.com/blog/resources/how-to-conv...   

                                              Link 3  
0  https://www.folio3.ai/blog/best-free-speech-to...  
1  https://www.ijert.org/research/implementation-...  
2  https://www.komando.com/downloads/convert-audi...  


### Exporting links

In [49]:
filepath = input("Enter filename: ")
filepath = filepath + '.xlsx'
with open(filepath, 'w+') as fp:
    pass
df.to_excel(filepath,index=True)

Enter filename: ReferenceLink_SpeechCon
